In [84]:
import numpy as np
import pandas as pd
import timeit

**This notebook captures different ways in which to filter lists with numeric and string data and compares the execution time of these methods.**

We will:
1. Generate a set of lists with different number of elements (1, 10, 100, 1_000, 10_000, 100_000)
2. Create different filter methods
3. Calculate the execution time for each and determine how much worse are the rest compared to the fastest (1-method time/fastest time)

**Helper functions**
*Plotting/presenting*

In [85]:
def style_negative(v, props=''):
    return props if v < 0 else None

def highlight_min(s, props=''):
    return np.where(s == np.nanmin(s.values), props, '')

def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

*Data creation*

In [86]:
def random_array(rows, columns):
    return np.random.rand(int(rows), columns)

a1 = random_array(1, 1)
a2 = random_array(10, 1)
a3 = random_array(100, 1)
a4 = random_array(1_000, 1)
a5 = random_array(10_000, 1)
a6 = random_array(100_000, 1)
a7 = random_array(1_000_000, 1)
a8 = random_array(10_000_000, 1)

# Example output
print(a1, a2, sep='\n\n')

[[0.70027988]]

[[0.35595219]
 [0.98875751]
 [0.23710326]
 [0.83522315]
 [0.04704935]
 [0.66543339]
 [0.28358301]
 [0.30530213]
 [0.35506463]
 [0.96086985]]


In [87]:
def random_list(np_array):
    return np_array.flatten().tolist()

d1 = random_list(a1)
d2 = random_list(a2)
d3 = random_list(a3)
d4 = random_list(a4)
d5 = random_list(a5)
d6 = random_list(a6)
d7 = random_list(a7)
d8 = random_list(a8)

# Example output
print(d1, d2, sep='\n\n')

[0.7002798831999606]

[0.35595218544510554, 0.9887575149309975, 0.23710326379088875, 0.8352231545173859, 0.04704934931326221, 0.6654333901308493, 0.2835830123003693, 0.305302133201766, 0.3550646307666714, 0.9608698498673375]


## Filter methods

In [88]:
class FilterMethodsNumeric:
    def __init__(self, list_, threshold=0.2):
        self.list_ = list_
        self.threshold = threshold

    def filter_for_loops(self):
        filtered_list = []
        for i in range(len(self.list_)):
            if self.list_[i] < self.threshold:
                filtered_list.append(self.list_[i])

        return filtered_list

    def filter_list_to_array_to_list(self):
        np_array = np.array(self.list_)
        np_array = np_array[np_array < self.threshold]
        return np_array.tolist()

    def filter_list_comprehension(self):
        return [x for x in self.list_ if x < self.threshold]

    def filter_filter(self):
        return list(filter(lambda x: x < self.threshold, self.list_))

    def filter_set(self):
        s = set(self.list_)
        return [x for x in s if x < self.threshold]

fm = FilterMethodsNumeric(list_=d2, threshold=0.2)

In [89]:
# Testing
fm.filter_list_comprehension()

[0.04704934931326221]

In [99]:
# fm.filter_filter()

In [100]:
# fm.filter_list_to_array_to_list()

In [98]:
# fm.filter_for_loops()

In [97]:
# fm.filter_set()

# Execution time summary
We display the final output as the ratio of each execution method vs the fastest method. Instead of looking at the absolute execution time, we prefer the relative time. This is because you might run the code in different machines, and it might be more interesting to know that method A was 3x slower, than to know it was 35 seconds slower.

In [95]:
results_dict = {'rows': [], 'for_loops': [], 'list_to_array_to_list': [], 'list_comprehension': [], 'filter': [], 'set': []}

n = 10

for ls in [d1, d2, d3, d4, d5, d6, d7, d8]:
    results_dict['rows'].append(len(ls))

    fm = FilterMethodsNumeric(list_=ls, threshold=0.2)
    results_dict['for_loops'].append(timeit.timeit(lambda: fm.filter_for_loops(), number=n))
    results_dict['list_to_array_to_list'].append(timeit.timeit(lambda: fm.filter_list_to_array_to_list(), number=n))
    results_dict['list_comprehension'].append(timeit.timeit(lambda: fm.filter_list_comprehension(), number=n))
    results_dict['filter'].append(timeit.timeit(lambda: fm.filter_filter(), number=n))
    results_dict['set'].append(timeit.timeit(lambda: fm.filter_set(), number=n))

results_df = pd.DataFrame(results_dict)
results_df.index = results_df['rows']
results_df = results_df.drop('rows', axis=1)

results_df['min_time_of_all_methods'] = results_df.min(axis=1)
results_df['for_loop_deterioration'] = (results_df['for_loops']/results_df['min_time_of_all_methods'])-1
results_df['array_list_deterioration'] = (results_df['list_to_array_to_list']/results_df['min_time_of_all_methods'])-1
results_df['list_comprehension_deterioration'] = (results_df['list_comprehension']/results_df['min_time_of_all_methods'])-1
results_df['filter_deterioration'] = (results_df['filter']/results_df['min_time_of_all_methods'])-1
results_df['set_deterioration'] = (results_df['set']/results_df['min_time_of_all_methods'])-1

In [96]:
# Execution time proportion between of method/fastest.
present_df = results_df[['for_loop_deterioration', 'array_list_deterioration','list_comprehension_deterioration', 'filter_deterioration', 'set_deterioration']].transpose()
present_df = (present_df.style.format(precision=2)
              .apply(highlight_min, props='color:white;background-color:darkgreen', axis=0)
              .apply(highlight_max, props='color:white;background-color:red', axis=0)
             )
present_df

rows,1,10,100,1000,10000,100000,1000000,10000000
for_loop_deterioration,0.30,0.72,1.16,1.41,1.74,1.52,1.50,1.27
array_list_deterioration,5.46,2.76,0.88,0.92,0.17,0.20,0.33,0.26
list_comprehension_deterioration,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
filter_deterioration,0.87,0.81,1.39,1.15,1.17,1.70,1.22,0.97
set_deterioration,0.46,0.71,5.68,1.55,1.68,3.04,5.68,4.28
